In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

model = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [2]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k=3,
    return_messages=True,
    memory_key="chat_history",
)


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)

human_prompt = HumanMessagePromptTemplate.from_template(
    "Please make emoji quiz of {movie}"
)
ai_prompt = AIMessagePromptTemplate.from_template("{answer}")

system_prompt = SystemMessagePromptTemplate.from_template(
    "You're an assistant who helps create emoji quizzes. If a person tells you the name of a movie, you represent it with three emojis. "
)

In [4]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate


examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie": "Tangled",
        "answer": "👸🧙‍💇‍‍",
    },
]

example_prompt = ChatPromptTemplate(messages=[human_prompt, ai_prompt])

few_shot_example = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=examples
)

In [5]:
final_prompt = ChatPromptTemplate(
    messages=[
        system_prompt,
        SystemMessagePromptTemplate.from_template(
            "Below is the example of the your answer format."
        ),
        few_shot_example,
        SystemMessagePromptTemplate.from_template("Below is the chat history"),
        MessagesPlaceholder(variable_name="chat_history"),
        SystemMessagePromptTemplate.from_template(
            "Below is the question of human, that you should answer it"
        ),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

In [6]:
from langchain.schema.runnable import RunnablePassthrough

chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | model


def invoke_chain_and_save_memory(question):
    result = chain.invoke({"question": question})
    return memory.save_context(
        {"input": question},
        {"output": result.content},
    )

In [12]:
invoke_chain_and_save_memory("Explain about Titanic")

🚢💔🎶

In [13]:
invoke_chain_and_save_memory("Explain about Avengers")

⚔️🦸‍♂️🌍

In [14]:
invoke_chain_and_save_memory("Explain about Man in Black")

👽🕶️🔫

In [15]:
invoke_chain_and_save_memory("What was the second movie I did ask?")

The second movie you asked about was "Avengers."